In [3]:
!pip install sagemaker -U

In [7]:
import sagemaker
import boto3
s=sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and s is not None:
    sagemaker_session_bucket=s.default_bucket()
try:
    role=sagemaker.get_execution_role()
except ValueError:
    iam=boto3.client('iam')
    role=iam.get_role(RoleName='sagemaker_execution')['Role']['Arn']
    s=sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {s.boto_region_name}")
    

sagemaker role arn: arn:aws:iam::167290362273:role/service-role/AmazonSageMaker-ExecutionRole-20251103T163046
sagemaker session region: ap-south-1


In [14]:
from sagemaker.huggingface.model import HuggingFaceModel

hub = {
  'HF_MODEL_ID':'distilbert-base-uncased-distilled-squad', # model_id from hf.co/models
  'HF_TASK':'question-answering'                           # NLP task you want to use for predictions
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,                                                # configuration for loading model from Hub
   role=role,                                              # IAM role with permissions to create an endpoint
   transformers_version="4.26",                             # Transformers version used
   pytorch_version="1.13",                                  # PyTorch version used
   py_version='py39',                                      # Python version used
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!

In [17]:
data = {
"inputs": {
	"question": "What is the Data science?",
	"context": "It is a Interdesciplanary field."
	}
}

In [18]:
predictor.predict(data)


{'score': 0.5596941709518433,
 'start': 8,
 'end': 31,
 'answer': 'Interdesciplanary field'}